# Пример решения

In [2]:
import pandas as pd
import xarray
import requests
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm_notebook as tqdm

## NCEP Dataset

Погодные данные из проекта [NCEP Reanalysis 2](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.html) — усреднённые за день температура воздуха, относительная влажность и компоненты ветра. Данные можно получить с 1979 года.

Загрузите наборы данных в каталог `data/ncep/`:
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/air.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/uwnd.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/rhum.2018.nc

In [4]:
ncep_data = []
for year in range(2016,2020):
    for var in ('air', 'uwnd', 'rhum'):
        dataset_filename = 'data/ncep/{}.{}.nc'.format(var, year)
        ncep_data.append(xarray.open_dataset(dataset_filename))
ncep_data = xarray.merge(ncep_data)

C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'uwnd' has multi

## Набор признаков на основе данных NCEP

Ищем наиболее близкий к точке узел сетки в наборе NCEP, в качестве признаков значения переменных зарегистрированные в день регистрации точки и агрегированные показатели за период от 1 до 3х недель до момента регистрации точки.

In [5]:
def extract_features(row):
    point = ncep_data.sel(
        lon=row['longitude'],
        lat=row['latitude'],
        level=1000,
        method='nearest',
    )
    

    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()
    p3w = point.rolling(time=21).mean()
    
    date = datetime.strptime(row['date'], '%Y-%m-%d').date()
    
    date1 = date + timedelta(days=1)
    date2 = date + timedelta(days=2)
    date3 = date + timedelta(days=3)
    
    date4 = date - timedelta(days=1)
    date5 = date - timedelta(days=2)
    date6 = date - timedelta(days=3)
    
    v = point.sel(time=datetime.strftime(date, '%Y-%m-%d'))
    v1w = p1w.sel(time=datetime.strftime(date, '%Y-%m-%d'))
    v2w = p2w.sel(time=datetime.strftime(date, '%Y-%m-%d'))
    v3w = p3w.sel(time=datetime.strftime(date, '%Y-%m-%d'))
    
    v1w1 = p1w.sel(time=datetime.strftime(date1, '%Y-%m-%d'))
    v2w1 = p2w.sel(time=datetime.strftime(date1, '%Y-%m-%d'))
    v3w1 = p3w.sel(time=datetime.strftime(date1, '%Y-%m-%d'))
    
    v1w2 = p1w.sel(time=datetime.strftime(date2, '%Y-%m-%d'))
    v2w2 = p2w.sel(time=datetime.strftime(date2, '%Y-%m-%d'))
    v3w2 = p3w.sel(time=datetime.strftime(date2, '%Y-%m-%d'))
    
    v1w3 = p1w.sel(time=datetime.strftime(date3, '%Y-%m-%d'))
    v2w3 = p2w.sel(time=datetime.strftime(date3, '%Y-%m-%d'))
    v3w3 = p3w.sel(time=datetime.strftime(date3, '%Y-%m-%d'))
    
    #------------------------------------------------------------ -
    
    v1w4 = p1w.sel(time=datetime.strftime(date4, '%Y-%m-%d'))
    v2w4 = p2w.sel(time=datetime.strftime(date4, '%Y-%m-%d'))
    v3w4 = p3w.sel(time=datetime.strftime(date4, '%Y-%m-%d'))
    
    v1w5 = p1w.sel(time=datetime.strftime(date5, '%Y-%m-%d'))
    v2w5 = p2w.sel(time=datetime.strftime(date5, '%Y-%m-%d'))
    v3w5 = p3w.sel(time=datetime.strftime(date5, '%Y-%m-%d'))
    
    v1w6 = p1w.sel(time=datetime.strftime(date6, '%Y-%m-%d'))
    v2w6 = p2w.sel(time=datetime.strftime(date6, '%Y-%m-%d'))
    v3w6 = p3w.sel(time=datetime.strftime(date6, '%Y-%m-%d'))
    
    return {
        'fire_id': row['fire_id'],
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'date': row['date'], 
        'temperature': v.air.values.item(0),
        'humidity': v.rhum.values.item(0),
        'uwind': v.uwnd.values.item(0),
        't1w': v1w.air.values.item(0),
        't2w': v2w.air.values.item(0),
        't3w': v3w.air.values.item(0),
        't1w1': v1w1.air.values.item(0),
        't2w1': v2w1.air.values.item(0),
        't3w1': v3w1.air.values.item(0),
        't1w2': v1w2.air.values.item(0),
        't2w2': v2w2.air.values.item(0),
        't3w2': v3w2.air.values.item(0),
        't1w3': v1w3.air.values.item(0),
        't2w3': v2w3.air.values.item(0),
        't3w3': v3w3.air.values.item(0),
        't1w4': v1w4.air.values.item(0),
        't2w4': v2w4.air.values.item(0),
        't3w4': v3w4.air.values.item(0),
        't1w5': v1w5.air.values.item(0),
        't2w5': v2w5.air.values.item(0),
        't3w5': v3w5.air.values.item(0),
        't1w6': v1w6.air.values.item(0),
        't2w6': v2w6.air.values.item(0),
        't3w6': v3w6.air.values.item(0),
        'h1w': v1w.rhum.values.item(0),
        'h2w': v2w.rhum.values.item(0),
        'h3w': v3w.rhum.values.item(0),
        'h1w1': v1w.rhum.values.item(0),
        'h2w1': v2w.rhum.values.item(0),
        'h3w1': v3w.rhum.values.item(0),
        'h1w2': v1w.rhum.values.item(0),
        'h2w2': v2w.rhum.values.item(0),
        'h3w2': v3w.rhum.values.item(0),
        'h1w3': v1w.rhum.values.item(0),
        'h2w3': v2w.rhum.values.item(0),
        'h3w3': v3w.rhum.values.item(0),
        'h1w4': v1w.rhum.values.item(0),
        'h2w4': v2w.rhum.values.item(0),
        'h3w4': v3w.rhum.values.item(0),
        'h1w5': v1w.rhum.values.item(0),
        'h2w5': v2w.rhum.values.item(0),
        'h3w5': v3w.rhum.values.item(0),
        'h1w6': v1w.rhum.values.item(0),
        'h2w6': v2w.rhum.values.item(0),
        'h3w6': v3w.rhum.values.item(0),
    }

## Выборка для обучения

In [10]:
import multiprocessing as mlp

In [13]:
df_train = pd.read_csv('data/wildfires_train.csv')
df_subsample = df_train.query('(date > "2017") & (date < "2020")')

df_features = []

for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
    features = extract_features(row)
    df_features.append(features)
df_features = pd.DataFrame(df_features)
df_features.iloc[:,3:].to_csv("features.csv")
df_features.set_index('fire_id', inplace=True)

C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


KeyboardInterrupt: 

## Обучение классификатора

,fire_type,fire_type_name,date,temperature,humidity,uwind,t1w,t2w,t3w,t1w1,...,h3w3,h1w4,h2w4,h3w4,h1w5,h2w5,h3w5,h1w6,h2w6,h3w6
fire_id,,,,,,,,,,,,,,,,,,,,,
166255,6,неконтролируемый пал,2019-04-10,277.309998,78.729996,-1.830002,277.309998,277.526672,276.237976,281.089996,...,77.707993,78.729996,75.273331,77.707993,78.729996,75.273331,77.707993,78.729996,75.273331,77.707993
173208,11,не подтверждено,2019-04-24,289.149994,47.020004,0.589996,289.149994,287.063324,285.019989,290.799988,...,37.348007,47.020004,40.850010,37.348007,47.020004,40.850010,37.348007,47.020004,40.850010,37.348007
172422,8,лесной пожар,2019-04-22,277.500000,42.140015,2.349991,277.500000,277.019989,276.846008,278.289978,...,66.892006,42.140015,55.386669,66.892006,42.140015,55.386669,66.892006,42.140015,55.386669,66.892006
161458,2,техногенный пожар,2019-03-08,276.250000,36.420013,3.330002,276.250000,273.579987,274.682007,277.549988,...,43.278008,36.420013,42.563343,43.278008,36.420013,42.563343,43.278008,36.420013,42.563343,43.278008
169367,10,контролируемый пал,2019-04-16,274.630005,45.980011,2.339996,274.630005,274.253326,277.273987,275.649994,...,60.616005,45.980011,58.970001,60.616005,45.980011,58.970001,60.616005,45.980011,58.970001,60.616005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169711,6,неконтролируемый пал,2019-04-17,274.799988,51.009995,-4.070007,274.799988,277.079987,277.356018,275.330017,...,61.135998,51.009995,52.616665,61.135998,51.009995,52.616665,61.135998,51.009995,52.616665,61.135998
162154,10,контролируемый пал,2019-03-17,283.089996,33.190002,2.279999,283.089996,281.806641,280.187988,276.959991,...,47.975998,33.190002,41.889999,47.975998,33.190002,41.889999,47.975998,33.190002,41.889999,47.975998
170361,5,сжигание мусора,2019-04-19,275.520020,40.350006,5.139999,275.520020,270.816681,271.798004,275.959991,...,51.116001,40.350006,49.836670,51.116001,40.350006,49.836670,51.116001,40.350006,49.836670,51.116001


In [31]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [32]:
X = df_features.iloc[:, 3:].fillna(0)
y = df_features['fire_type']

In [41]:
fire_classifier = GradientBoostingClassifier(max_depth=30,
                                            learning_rate=0.01)

In [40]:
cross_val_score(
    fire_classifier, 
    X, y, 
    cv=10, 
)

C:\Users\Xiaomi Pro\Anaconda3\envs\geoenv\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [36]:
fire_classifier.fit(X, y)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Решение для отправки в систему

In [37]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(fire_classifier, fout, protocol=pickle.HIGHEST_PROTOCOL)

## Прогнозирование на новых данных

In [38]:
df_predictions = pd.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)
df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
fire_id,,,,,,,,,,
166255,0.000020,0.000333,0.007068,0.050796,0.003896,0.322829,0.004353,0.286868,0.200400,0.123437
173208,0.000004,0.000258,0.002668,0.004108,0.001298,0.067067,0.008345,0.377398,0.029063,0.509790
172422,0.000004,0.000111,0.003288,0.010116,0.001284,0.167586,0.438420,0.233723,0.081186,0.064283
161458,0.000002,0.315392,0.001763,0.002575,0.000600,0.561428,0.013131,0.040349,0.041895,0.022864
169367,0.000006,0.000043,0.007493,0.010941,0.001569,0.502012,0.008688,0.056127,0.379539,0.033583


In [82]:
df_predictions = pandas.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)
df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
fire_id,,,,,,,,,,
164359,4.426583e-07,0.000017,0.004756,0.021454,0.000290,0.051475,0.002226,0.513595,0.207536,0.198651
161022,2.848951e-07,0.000199,0.000937,0.009518,0.000182,0.693501,0.125599,0.058077,0.084669,0.027317
164678,6.347527e-07,0.000035,0.007284,0.012029,0.000385,0.190829,0.020084,0.545620,0.120056,0.103677
165757,5.220913e-07,0.000039,0.004668,0.025462,0.000459,0.216522,0.002104,0.353030,0.096606,0.301110
160358,3.671328e-07,0.000007,0.252453,0.022867,0.001219,0.259000,0.008091,0.217179,0.103729,0.135455
